# Introdução

# Dependências

In [2]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install tweepy
!{sys.executable} -m pip install requests

You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
You are using pip version 9.0.3, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
import pandas as pd
import numpy as np
import requests
import tweepy

import json
import os

# Coleta

In [2]:
predictions_url = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'
filename = 'image_predictions.tsv'
if not os.path.exists(filename):
    response = requests.get(predictions_url)
    with open(filename, 'wb') as file:
        file.write(response.content)

In [11]:
df_predictions = pd.read_csv('image_predictions.tsv', sep='\t')
df_predictions_clean = df_predictions.copy()
df_predictions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
tweet_id    2075 non-null int64
jpg_url     2075 non-null object
img_num     2075 non-null int64
p1          2075 non-null object
p1_conf     2075 non-null float64
p1_dog      2075 non-null bool
p2          2075 non-null object
p2_conf     2075 non-null float64
p2_dog      2075 non-null bool
p3          2075 non-null object
p3_conf     2075 non-null float64
p3_dog      2075 non-null bool
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [13]:
df_tweets = pd.read_csv('twitter-archive-enhanced.csv')
df_tweets_clean = df_tweets.copy()
df_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
tweet_id                      2356 non-null int64
in_reply_to_status_id         78 non-null float64
in_reply_to_user_id           78 non-null float64
timestamp                     2356 non-null object
source                        2356 non-null object
text                          2356 non-null object
retweeted_status_id           181 non-null float64
retweeted_status_user_id      181 non-null float64
retweeted_status_timestamp    181 non-null object
expanded_urls                 2297 non-null object
rating_numerator              2356 non-null int64
rating_denominator            2356 non-null int64
name                          2356 non-null object
doggo                         2356 non-null object
floofer                       2356 non-null object
pupper                        2356 non-null object
puppo                         2356 non-null object
dtypes: float64(4), int64(3), ob

# Contagem de retweets e favoritos

In [5]:
consumer_key = os.environ['TWITTER_CONSUMER_KEY']
consumer_secret = os.environ['TWITTER_CONSUMER_SECRET']
access_token = os.environ['TWITTER_ACCESS_TOKEN']
access_token_secret = os.environ['TWITTER_ACCESS_TOKEN_SECRET']

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

twitter_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [24]:
def get_retweet_fav_counts(row):
    tweet_id = row['tweet_id']
    try:
        tweet_status = twitter_api.get_status(row['tweet_id'], tweet_mode='extended')
        return (tweet_id, tweet_status.retweet_count, tweet_status.favorite_count)
    except tweepy.TweepError:
        return (tweet_id, None, None)

In [38]:
filename = 'tweet_counts.csv'
if not os.path.exists(filename):
    tweet_counts = df_tweets_clean.apply(lambda row: get_retweet_fav_counts(row), axis=1)
    df_tweet_counts = pd.DataFrame.from_records(list(tweet_counts), columns=['tweet_id', 'retweet_count', 'favorite_count'])
    df_tweet_counts.to_csv(filename, index=False)
else:
    df_tweet_counts = pd.read_csv(filename)

In [39]:
df_tweet_counts.sample(3)

,tweet_id,retweet_count,favorite_count
1443,696744641916489729,1028.0,2217.0
790,773922284943896577,1923.0,6918.0
1779,677716515794329600,1056.0,3223.0


# Avaliação

# Limpeza